In [123]:
from dotenv import load_dotenv
import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import sys

In [128]:
# api connection
load_dotenv()
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
URI = os.getenv("URI")

sp_oauth = SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID,
                                               client_secret=SPOTIFY_CLIENT_SECRET,
                                               redirect_uri=URI,
                                               scope="user-library-read user-top-read user-read-recently-played")

token_info = sp_oauth.get_cached_token()
#sp = spotipy.Spotify(auth=token_info["access_token"])

In [62]:
top_tracks = sp.current_user_top_tracks()
#print(top_tracks["items"])
#del top_tracks["items"]["albums"]["available_markets"]
#print(top_tracks["items"][-1])
df_top = pd.DataFrame(top_tracks["items"])
to_drop = ["available_markets", "available_markets", "href", "external_urls", "disc_number", "track_number", "external_ids", "preview_url", "uri", "is_local", 'id', 'is_playable', 'type', "album", 'artists']
df_top = df_top.drop(columns=to_drop)
print(df_top.columns)

# Convert milliseconds to seconds
df_top['duration_sec'] = df_top['duration_ms'] / 1000

# Calculate minutes and seconds
df_top['duration_min'] = df_top['duration_sec'] // 60
df_top['duration_sec'] = df_top['duration_sec'] % 60

# Format as 'mm:ss'
df_top['duration_mm_ss'] = df_top['duration_min'].astype(int).astype(str) + ':' + df_top['duration_sec'].astype(int).astype(str).str.zfill(2)

print(df_top.head(5))

Index(['duration_ms', 'explicit', 'name', 'popularity'], dtype='object')
   duration_ms  explicit         name  popularity  duration_sec  duration_min  \
0       223192      True         Juno          89        43.192           3.0   
1       267866      True      Goddess          69        27.866           4.0   
2       225360     False     Lovesick          71        45.360           3.0   
3       169917     False         APT.          92        49.917           2.0   
4       225500     False  Without You          53        45.500           3.0   

  duration_mm_ss  
0           3:43  
1           4:27  
2           3:45  
3           2:49  
4           3:45  


In [109]:
print("Total tracks listened: ", len(df_top["name"]))

total_ms = df_top["duration_ms"].sum()
total_sec = total_ms / 1000
total_min = total_sec // 60
total_sec = (total_sec % 60).astype(int)

total_hr = (total_min // 60).astype(int)
total_min = (total_min % 60).astype(int)

print("total min: ", total_min)
print("total_sec: ", total_sec)
print("total hrs: ", total_hr)

print("Total time listend to: ", str(total_hr) + ":0" + str(total_min) +  "." + str(total_sec))

Total tracks listened:  20
total min:  9
total_sec:  27
total hrs:  1
Total time listend to:  1:09.27
